In [6]:
from sage.all import *
from Crypto.Util.number import long_to_bytes
import itertools

n = 147411106158287041622784593501613775873740698182074300197321802109276152016347880921444863007
e = 3
c = 114267757154492856513993877195962986022489770009120200367811440852965239059854157313462399351

# 素因子列表
primes = [
    8532679, 9613003, 9742027, 10660669, 11451571, 
    11862439, 11908471, 13164721, 13856221, 14596051, 
    15607783, 15840751, 16249801
]

roots_per_prime = []
print("[*] 正在计算每个素数模下的立方根...")

for i, p in enumerate(primes):
    # 在有限域 GF(p) 下求 e 次方根
    # nth_root(e, all=True) 会返回该模下所有可能的根
    try:
        # 转化为 Sage 的整数类型
        p = Integer(p)
        c_mod = Mod(c, p)
        
        # 求解 x^3 = c mod p
        # nth_root 返回所有根的列表
        roots = c_mod.nth_root(e, all=True)
        
        # 将结果转回普通整数，方便后续处理
        roots = [Integer(r) for r in roots]
        
        print(f"    Prime {p} (mod 3 = {p%3}): Found {len(roots)} roots.")
        
        if len(roots) == 0:
            print(f"[!] 错误：在素数 {p} 下无解，无法继续。")
            exit()
            
        roots_per_prime.append(roots)
        
    except Exception as err:
        print(f"[!]计算出错: {err}")
        exit()

# 组合爆破 (CRT)
total_combinations = 1
for r in roots_per_prime:
    total_combinations *= len(r)

print("[*] 开始爆破组合并筛选 Flag...")

count = 0
found_flag = False

# 使用 itertools.product 生成所有根的组合
for roots_comb in itertools.product(*roots_per_prime):
    count += 1
    
    # CRT_list 是 Sage 内置函数，比手动 crt 快且方便
    # x = r1 mod p1, x = r2 mod p2 ...
    m = CRT_list(list(roots_comb), primes)
    
    # 转为 bytes
    m_bytes = long_to_bytes(m)
    try:
        # 尝试解码为 ASCII
        m_str = m_bytes.decode('ascii')
        
        # 检查是否全部由可打印字符组成
        if all(0x20 <= ord(c) <= 0x7e for c in m_str):
            print(f"[+] Found Candidate ({count}/{total_combinations}):")
            print(f"    Str: {m_str}")
            
            if "CTF{" or "ctf{" in m_str: found_flag = True; break
            
    except UnicodeDecodeError:
        # 包含乱码，通常不是 Flag
        pass

print("[*] 完成。")

[*] 正在计算每个素数模下的立方根...
    Prime 8532679 (mod 3 = 1): Found 3 roots.
    Prime 9613003 (mod 3 = 1): Found 3 roots.
    Prime 9742027 (mod 3 = 1): Found 3 roots.
    Prime 10660669 (mod 3 = 1): Found 3 roots.
    Prime 11451571 (mod 3 = 1): Found 3 roots.
    Prime 11862439 (mod 3 = 1): Found 3 roots.
    Prime 11908471 (mod 3 = 1): Found 3 roots.
    Prime 13164721 (mod 3 = 1): Found 3 roots.
    Prime 13856221 (mod 3 = 1): Found 3 roots.
    Prime 14596051 (mod 3 = 1): Found 3 roots.
    Prime 15607783 (mod 3 = 1): Found 3 roots.
    Prime 15840751 (mod 3 = 1): Found 3 roots.
    Prime 16249801 (mod 3 = 1): Found 3 roots.
[*] 开始爆破组合并筛选 Flag...
[+] Found Candidate (104964/1594323):
    Str: BCCTF{y0U_h4V3_g0T_70_b3_K1DD1n9_Me}
[*] 完成。
